In [1]:
# Import pandas
import pandas as pd
import numpy as np
from gensim.corpora import Dictionary
from sklearn.model_selection import train_test_split

In [2]:
# Assign spreadsheet filename to `file`
file = 'Eclipse_4sourcev1.xls'
# Load spreadsheet
xl = pd.ExcelFile(file)
# Load a sheet into a DataFrame by name: df1
df = xl.parse('total')

In [3]:
print('Total: ', len(df))

Total:  7373


In [4]:
class NaiveBayesClassifer:
    def __init__(self):
        self.name = 'Naive Bayes Classifier'
    
    def train (self, X_train, y_train, number_of_unique_words = 100000):
        self.prior = np.zeros(5)
        labels = list(y_train)
        
        for i in range(5):
            self.prior[i] = labels.count(i)/len(labels)
        print(self.prior)
        self.mle_summary = np.zeros((number_of_unique_words, 5))
        self.wordCount_summary = np.zeros(5)
        
        summaries = list(X_train.loc[:,'summary'])

        self.dictionary = Dictionary()

        for summary in summaries:
            self.dictionary.add_documents([summary.split()])

        for i in range(len(summaries)):
            summaries[i] = self.dictionary.doc2idx(summaries[i].split())
        
        for i in range(len(summaries)):
            summary = summaries[i]
            for word in summary:
                self.mle_summary[word][labels[i]] += 1 
                self.wordCount_summary[labels[i]] += 1
        
        for i in range(number_of_unique_words):
            for j in range(5):
                self.mle_summary[i][j] += 1
                self.mle_summary[i][j] /= (self.wordCount_summary[j])
        
        self.mle_desc = np.zeros((number_of_unique_words, 5))
        self.wordCount_desc = np.zeros(5)
        
        descriptions = list(X_train.loc[:,'description'])
    
        for description in descriptions:
            self.dictionary.add_documents([description.split()])

        for i in range(len(descriptions)):
            descriptions[i] = self.dictionary.doc2idx(descriptions[i].split())
        
        
        for i in range(len(descriptions)):
            description = descriptions[i]
            for word in description:
                '''
                if (i==0):
                    print(word, self.wordCount_desc[word])
                '''
                self.mle_desc[word][labels[i]] += 1
                self.wordCount_desc[labels[i]] += 1
        
        for i in range(number_of_unique_words):
            for j in range(5):
                self.mle_desc[i][j] += 1
                self.mle_desc[i][j] /= self.wordCount_desc[labels[j]]
        
        pass
    
    def predict (self, X_test):
        y_test = np.ones((X_test.shape[0], 5))
        for i in range(X_test.shape[0]):
            for j in range(5):
                y_test[i][j] = (self.prior[j])
        
        summaries = list(X_test.loc[:,'summary'])
        for summary in summaries:
            self.dictionary.add_documents([summary.split()])
        for i in range(len(summaries)):
            summaries[i] = self.dictionary.doc2idx(summaries[i].split())
        
        descriptions = list(X_test.loc[:,'description'])
        for description in descriptions:
            self.dictionary.add_documents([description.split()])
        for i in range(len(descriptions)):
            descriptions[i] = self.dictionary.doc2idx(descriptions[i].split())
        
        for i in range(X_test.shape[0]):
            for j in range(5):
                log_sum_summary = 0
                log_sum_desc = 0
                for word in summaries[i]:
                    if word != -1 and self.mle_summary[word][j] != 0:
                        log_sum_summary += np.log(self.mle_summary[word][j])
                for word in descriptions[i]:
                    if word != -1 and self.mle_desc[word][j] != 0:
                        log_sum_desc += np.log(self.mle_desc[word][j])
                
                y_test[i][j] = np.log(y_test[i][j])
                y_test[i][j] += ((log_sum_desc) )
                
                ''' + (log_sum_desc) '''
        #print(y_test)
        ret_val = np.zeros(len(y_test))
        
        cnt = 0
        for i in range(X_test.shape[0]):
            mx = 0
            mxindex = -1
            for j in range(5):
                if y_test[i][j] != 0:
                    if mxindex == -1 or y_test[i][j] > mx:
                        mxindex = j
                        mx = y_test[i][j]
            if mxindex == -1:
                cnt += 1
                mxindex = cnt % 5
            ret_val[i] = mxindex
        ret_val =  [int(round(x)) for x in ret_val] 
        print('Number of random classifications: ',cnt)
        return list(ret_val)

In [5]:

# create training and testing vars
y = df.loc[:,'label'] - 1
X = df.loc[:, 'summary':'component']

In [6]:
naiveBayesClassifier = NaiveBayesClassifer()
f1_scores_list = []

In [8]:
f1_scores = np.zeros(5)
for i in range(1,11):
    split_point = (len(X)/11) * i
    naiveBayesClassifier.train(X_train = X.loc[0:split_point,], y_train = y.loc[0:split_point, ])
    y_test = naiveBayesClassifier.predict(X_test = X.loc[split_point:, ])
    
    
    y_label = list(y.loc[split_point:, ])
    
    #print(y_test, y_label)
    #print((y_test == y_label).count(True) / len(y_test))
    
    
    true_positive = np.zeros(5)
    
    false_positive = np.zeros(5)
    false_negative = np.zeros(5)
    
    for j in range(len(y_test)):
        if (y_test[j] == y_label[j]):
            true_positive[y_test[j]] += 1
        else:
            false_positive[y_test[j]] += 1
            false_negative[y_label[j]] += 1
    
    precision = np.zeros(5)
    recall = np.zeros(5)
    for j in range(5):
        precision[j] = true_positive[j] / (true_positive[j] + false_positive[j])
        recall[j] = true_positive[j] / (true_positive[j] + false_negative[j])
    
    for j in range(5):
        f1_scores[j] = (2 * (precision[j] * recall[j])) / (precision[j] + recall[j]) 
    print('Count: Train vs Test')
    print(y_test.count(0), y_label.count(0))
    print(y_test.count(1), y_label.count(1))
    print(y_test.count(2), y_label.count(2))
    print(y_test.count(3), y_label.count(3))
    print(y_test.count(4), y_label.count(4))
    
    print('F-measure after ', i, 'th iteration: ', f1_scores)
    f1_scores_list.append(f1_scores)
    pass
    

[0.04769001 0.3681073  0.33532042 0.17585693 0.07302534]
Number of random classifications:  0
Count: Train vs Test
0 698
4889 1191
1804 2756
8 1268
1 789
F-measure after  1 th iteration:  [       nan 0.30723684 0.29692982 0.00470219        nan]
[0.05070843 0.31618195 0.35868755 0.18866518 0.0857569 ]


/home/immuntasir/.local/lib/python3.5/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars
/home/immuntasir/.local/lib/python3.5/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars


Number of random classifications:  0
Count: Train vs Test
0 662
2352 1014
3674 2500
6 1133
0 723
F-measure after  2 th iteration:  [       nan 0.28282828 0.47748623 0.00351185        nan]
[0.0512183  0.27747389 0.37493784 0.18896072 0.10740925]
Number of random classifications:  0
Count: Train vs Test
0 627
331 880
5026 2227
5 1006
0 622
F-measure after  3 th iteration:  [       nan 0.12881916 0.57410727 0.00197824        nan]
[0.05033557 0.24794929 0.39932886 0.19686801 0.10551827]
Number of random classifications:  0
Count: Train vs Test
0 595
48 773
4642 1910
1 858
0 555
F-measure after  4 th iteration:  [       nan 0.02679659 0.57814408        nan        nan]
[0.0528043  0.23687351 0.40662291 0.19570406 0.10799523]
Number of random classifications:  0
Count: Train vs Test
0 553
28 644
3993 1618
0 730
0 476
F-measure after  5 th iteration:  [       nan 0.02380952 0.57387275        nan        nan]
[0.05768274 0.22501243 0.41074092 0.20014918 0.10641472]
Number of random classificatio

In [9]:
np.mean(np.array(f1_scores_list), axis = 0)

array([       nan,        nan, 0.56129032,        nan,        nan])